# lg.ipynb

## 📌 `lg_hybrid_grade.py` RAG 시스템 동작 흐름

### 1️⃣ 사용자 입력

* 사용자가 질문을 입력한다.

---

### 2️⃣ 하이브리드 검색 (`retriever` 노드)

* 벡터 검색(임베딩 기반) 수행
* BM25 검색(키워드 기반) 수행
* `get_doc_hash_key(context + source)`을 사용하여 문서별 **고유 키 생성**
* 각 검색 결과(벡터, BM25)에 고유 키를 매핑하여 문서 중복 제거
* 두 검색 방식에서 모두 조회된 문서는 **RRF(Reciprocal Rank Fusion) 점수 방식으로 점수 누적**

> **RRF 공식:** `1 / (K + rank + 1)`
> **K = 60**

➡ 누적 RRF 점수를 기준으로 **상위 3개의 문서를 선택하여 grade 노드로 전달**

---

### 3️⃣ 문서 평가 (`grade_node`) ← **중요 단계**

* 선택된 문서 3개를 **relevance score 또는 LLM 평가 기반으로 필터링**
* 예: `score ≥ 0.7` 인 문서만 유지

📌 (예정된 수정 방향)

* 기존 "3개만 선택" 방식 → "문서 평가(점수 기반 필터링)" 방식으로 개선하여 **정확도 및 잡음 문서 제거 강화**

---

### 4️⃣ 조건 분기 (`decide_to_generate`)

| 조건           | 다음 단계               |
| ------------ | ------------------- |
| 유지된 문서 수 ≥ 1 | `generate` 노드로 이동   |
| 유지된 문서 수 = 0 | `web_search` 노드로 이동 |

---

### 5️⃣ 웹 검색 (`web_search`, Tavily API)

* 내부 문서 DB에서 답변이 불가능한 경우 **인터넷 검색 기반 문서 수집**
* 검색 결과를 **Document 구조로 state에 저장**

📌 웹 검색 결과도 grade_node를 통해 점수 기반 필터링을 수행
→ **필터링 후 문서가 존재하면 generate로 이동**

⚠ 필터링 후 문서가 1개도 없을 경우:

> "조회할 수 없습니다." 메시지 반환 (환각 방지 정책)

---

### 6️⃣ 답변 생성 (`generate`)

| 문서 상태               | 답변 생성 방식             |
| ------------------- | -------------------- |
| 내부 문서 있음            | 내부 DB 문서를 기반으로 답변 생성 |
| 내부 문서 없음 (웹 검색 문서만) | 웹 검색 문서를 기반으로 답변 생성  |

* 프롬프트는 **문서 기반 답변만 허용** → 임의 생성 / 날조 방지

---

### 7️⃣ 사용자 응답

* 최종 생성된 답변을 사용자에게 전달

---

### ⭐ 특징

| 목표        | 지원 방식                    |
| --------- | ------------------------ |
| 검색 정확도 향상 | BM25 + 벡터 + RRF 융합       |
| 불필요 문서 제거 | LLM/score 기반 grade 단계    |
| 환각 최소화    | 문서 기반 생성 + 문서 0개 시 안전 응답 |
| 정보 누락 보완  | 내부 DB 실패 시 웹 검색 fallback |


# lg_hybrid_rerank.py

## 📌 `lg_hybrid_rerank.py` RAG 시스템 동작 흐름

### 1️⃣ 사용자 입력

* 사용자가 질문을 입력한다.

---

### 2️⃣ 하이브리드 검색 (`retriever` 노드)

* 벡터 검색(임베딩 기반) 수행
* BM25 검색(키워드 기반) 수행
* 문서 내용 + source(URL)로 고유 키 생성하여 **중복 문서 제거**
* 두 검색 방식에 모두 포함된 문서는 **RRF 점수 방식으로 점수 누적**

> **RRF 공식:** `1 / (K + rank + 1)`
> **K = 60**

➡ RRF 점수를 기준으로 **상위 3개의 문서를 최종 후보 문서로 선택**

---

### 3️⃣ 문서 재정렬 (`rerank_documents_node`)

* 후보 문서 3개를 질문과의 관련도 기준으로 **다시 평가**
* 평가 방식: LLM 기반 평가 또는 별도의 reranking 모델

---

### 4️⃣ 조건 분기 (`decide_to_generate`)

| 조건             | 다음 단계               |
| -------------- | ------------------- |
| 최종 문서 1개 이상 존재 | `generate` 노드로 이동   |
| 문서가 0개         | `web_search` 노드로 이동 |

---

### 5️⃣ 웹 검색 (`web_search`, Tavily API)

* 내부 DB에서 답을 찾지 못한 경우 **웹에서 문서를 수집**
* 수집된 문서를 **Document 구조로 state에 저장**

---

### 6️⃣ 문서 평가 (`grade_documents_node`) ← **신설 단계**

* 웹 검색 결과 문서를 relevance score 또는 LLM 평가 스코어 기준으로 **필터링**
* 예: `score ≥ 0.7` 인 문서만 유지

📌 **주의**

* 필터링 후 문서가 1개도 남지 않을 경우:
  → `generate` 단계로 가면 할루시네이션 발생 위험이 매우 높음
  → 현재 정책: **"조회할 수 없습니다."** 를 반환하여 안전 종료

---

### 7️⃣ 답변 생성 (`generate`)

| 문서 상태    | 답변 생성 방식            |
| -------- | ------------------- |
| 내부 문서 존재 | 내부 문서를 기반으로 답변 생성   |
| 내부 문서 없음 | 웹 검색 문서를 기반으로 답변 생성 |

* 프롬프트는 **문서 기반 답변만 허용**하도록 구성되어 환각을 최소화

---

### 8️⃣ 사용자 응답

* 최종 생성된 답변이 사용자에게 전달된다.

---

### ✨ 전체 요약

* 하이브리드 검색으로 문서 선택 → Rerank → 필요 시 웹 검색 → 문서 재평가 → 안전 기반 생성
* **핵심 목표: 검색 성능 + 안전성 + 환각 방지**

```
┌──────────────────────────────┐
│            START              │
└──────────────────────────────┘
                │
                ▼
┌──────────────────────────────────────┐
│  Retriever  (Hybrid: BM25 + Vector)  │
└──────────────────────────────────────┘
                │
                ▼
┌──────────────────────────────────────┐
│        Rerank Documents Node         │
└──────────────────────────────────────┘
                │
                ▼
┌──────────────────────────────────────┐
│          decide_to_generate          │
└──────────────────────────────────────┘
        ┌───────────────┴───────────────┐
        │                               │
        ▼                               ▼
┌────────────────────────────┐   ┌────────────────────────────┐
│          generate           │   │          web_search         │
└────────────────────────────┘   └────────────────────────────┘
                                        │
                                        ▼
                     ┌────────────────────────────────────────┐
                     │        grade_documents_node             │
                     │  (문서 필터링: threshold 기반)          │
                     └────────────────────────────────────────┘
                                        │
                                        ▼
                     ┌────────────────────────────────────────┐
                     │               generate                  │
                     └────────────────────────────────────────┘
                                        │
                                        ▼
┌─────────────────────────────────────────────┐
│                     END                      │
└─────────────────────────────────────────────┘
```